In [2]:
# 🧪 Install required packages (quiet mode)
!pip install -q requests matplotlib pandas pyinputplus ipywidgets

# ✅ Imports
import requests
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import re
import pyinputplus as pyip
from typing import Dict, Any
import ipywidgets as widgets
from IPython.display import display, clear_output

# ✅ Geocoding using OpenStreetMap Nominatim
def geocode_location(location: str) -> tuple[float, float]:
    url = "https://nominatim.openstreetmap.org/search"
    params = {'q': location, 'format': 'json', 'limit': 1}
    headers = {'User-Agent': 'weather-colab'}  # Required by Nominatim
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()
    if not data:
        raise ValueError(f"❌ Location '{location}' not found.")
    return float(data[0]['lat']), float(data[0]['lon'])

# ✅ Fetch weather data from Open-Meteo API
def fetch_weather(lat: float, lon: float, start_date: str, end_date: str) -> Dict[str, Any]:
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        'latitude': lat,
        'longitude': lon,
        'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum',
        'current_weather': True,
        'timezone': 'auto',
        'start_date': start_date,
        'end_date': end_date
    }
    return requests.get(url, params=params).json()

# ✅ Extract forecast data
def _extract_forecast_data(weather_data, keys):
    forecast = weather_data.get('forecast', [])
    return [[day.get(k, None) for day in forecast] for k in keys]

# ✅ Get full weather data
def get_weather_data(location: str, forecast_days: int = 5) -> Dict[str, Any]:
    forecast_days = min(max(forecast_days, 1), 5)
    lat, lon = geocode_location(location)
    today = datetime.date.today()
    end_date = today + datetime.timedelta(days=forecast_days - 1)
    raw_data = fetch_weather(lat, lon, today.isoformat(), end_date.isoformat())
    result = {
        'location': location,
        'latitude': lat,
        'longitude': lon,
        'current': raw_data.get('current_weather', {}),
        'forecast': []
    }
    daily = raw_data.get('daily', {})
    for date, t_max, t_min, precip in zip(
        daily.get('time', []),
        daily.get('temperature_2m_max', []),
        daily.get('temperature_2m_min', []),
        daily.get('precipitation_sum', [])
    ):
        result['forecast'].append({
            'date': date,
            'temp_max': t_max,
            'temp_min': t_min,
            'precipitation': precip
        })
    return result

# ✅ Plot temperature
def create_temperature_visualisation(weather_data, output_type='display'):
    dates, temp_max, temp_min = _extract_forecast_data(weather_data, ['date', 'temp_max', 'temp_min'])
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(dates, temp_max, label='Max Temp (°C)', marker='o', color='crimson')
    ax.plot(dates, temp_min, label='Min Temp (°C)', marker='o', color='royalblue')
    ax.fill_between(dates, temp_min, temp_max, color='lightgray', alpha=0.3)
    ax.set_title(f"Temperature Forecast for {weather_data.get('location', 'Unknown')}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Temperature (°C)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
    plt.xticks(rotation=45)
    plt.tight_layout()
    if output_type == 'figure':
        return fig
    else:
        plt.show()

# ✅ Plot precipitation
def create_precipitation_visualisation(weather_data, output_type='display'):
    forecast = weather_data.get('forecast', [])
    if forecast:
        dates, precipitation = zip(*[
            (day.get('date', 'Unknown'), day.get('precipitation') if day.get('precipitation') is not None else 0)
            for day in forecast if 'date' in day
        ])
    else:
        dates, precipitation = ([], [])
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(dates, precipitation, color='deepskyblue', width=0.6)
    ax.set_title(f"Precipitation Forecast for {weather_data.get('location', 'Unknown')}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Precipitation (mm)")
    ax.grid(axis='y', linestyle='--', alpha=0.5)
    plt.xticks(rotation=45)
    plt.tight_layout()
    if output_type == 'figure':
        return fig
    else:
        plt.show()

# ✅ Natural language parsing
def parse_weather_question(question):
    original = question
    question = question.lower()
    location = None
    time_period = 'today'
    weather_attribute = 'general'
    is_vague = False

    if 'tomorrow' in question:
        time_period = 'tomorrow'
    elif 'soon' in question or 'later' in question:
        time_period = 'next_1_days'
    else:
        match = re.search(r'next (\d+) days', question)
        if match:
            time_period = f"next_{match.group(1)}_days"

    if any(term in question for term in ['temperature', 'temp', 'hot', 'cold']):
        weather_attribute = 'temperature'
    elif any(term in question for term in ['rain', 'precipitation', 'snow', 'wet']):
        weather_attribute = 'precipitation'
    elif 'forecast' in question:
        weather_attribute = 'forecast'
    elif 'humidity' in question:
        weather_attribute = 'humidity'
    elif 'wind' in question:
        weather_attribute = 'wind'

    location_match = re.search(r'\bin ([a-z\s\-]+)', question)
    if location_match:
        location = location_match.group(1).strip()
    else:
        # fallback: find capitalized words as location
        caps = re.findall(r'\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)?', original)
        if caps:
            location = ' '.join(caps)

    if not location or weather_attribute == 'general':
        is_vague = True

    return {
        'location': location,
        'time_period': time_period,
        'weather_attribute': weather_attribute,
        'is_vague': is_vague
    }

# ✅ Generate a response
def generate_weather_response(parsed: dict, weather_data: dict) -> str:
    location = parsed['location']
    time = parsed['time_period']
    attr = parsed['weather_attribute']
    forecast = weather_data.get('forecast', [])
    current = weather_data.get('current', {})

    if parsed['is_vague']:
        return "❗ Please include a location and weather detail (e.g. temperature or rain)."

    # Helper: find forecast for a specific date
    def forecast_for_date(date_str):
        for day in forecast:
            if day['date'] == date_str:
                return day
        return None

    today = datetime.date.today()
    if time == 'today':
        target_date = today.isoformat()
    elif time == 'tomorrow':
        target_date = (today + datetime.timedelta(days=1)).isoformat()
    elif time.startswith('next_'):
        # For multi-day forecasts, just return summary
        days = int(time.split('_')[1])
        temps = [f"{d['date']}: {d['temp_min']}°C - {d['temp_max']}°C" for d in forecast[:days]]
        return "🌡️ Temperature forecast:\n" + "\n".join(temps)
    else:
        target_date = today.isoformat()

    day_forecast = forecast_for_date(target_date)
    if not day_forecast:
        return f"⚠️ No forecast data available for {target_date}."

    # Handle precipitation (rain/snow)
    if attr == 'precipitation':
        precip = day_forecast.get('precipitation', 0)
        if precip > 0:
            return f"🌧️ Yes, it looks like there will be precipitation ({precip} mm) in {location} on {target_date}."
        else:
            return f"❌ No, it doesn't look like it will rain or snow in {location} on {target_date}."

    # Handle temperature
    elif attr == 'temperature':
        return (f"🌡️ On {target_date} in {location}, the temperature will range from "
                f"{day_forecast['temp_min']}°C to {day_forecast['temp_max']}°C.")

    # Handle wind
    elif attr == 'wind':
        windspeed = current.get('windspeed', 'unknown')
        return f"💨 Wind speed in {location} is {windspeed} km/h."
    # Handle humidity (not available)
    elif attr == 'humidity':
        return "💧 Sorry, humidity data is not available from this API."

    # Generic fallback
    return "⚠️ Could not understand your weather request."

# ====== IPyWidgets UI Integration ======

# Widgets
location_input = widgets.Text(
    description='📍 Location:',
    placeholder='e.g., Sydney',
    style={'description_width': 'initial'}
)

question_input = widgets.Text(
    description='💬 Ask:',
    placeholder='e.g., Will it rain in Berlin tomorrow?',
    style={'description_width': 'initial'}
)

# Custom colored buttons
ask_button = widgets.Button(
    description='Ask Weather',
    button_style='',  # No predefined style
    layout=widgets.Layout(width='150px')
)
ask_button.style.button_color = '#3498db'  # Blue

temp_button = widgets.Button(
    description='Show Temperature',
    button_style='success',  # Green
    layout=widgets.Layout(width='150px')
)
# No custom color needed; 'success' is green

precip_button = widgets.Button(
    description='Show Precipitation',
    button_style='',  # No predefined style
    layout=widgets.Layout(width='180px')
)
precip_button.style.button_color = '#f39c12'  # Orange

output_area = widgets.Output()

# Handlers
def on_ask_clicked(b):
    with output_area:
        clear_output()
        question = question_input.value.strip()
        parsed = parse_weather_question(question)
        if not parsed['location']:
            print("❌ Please specify a location.")
            return
        try:
            data = get_weather_data(parsed['location'])
            print(generate_weather_response(parsed, data))
        except Exception as e:
            print(f"⚠️ {e}")

def on_temp_clicked(b):
    with output_area:
        clear_output()
        loc = location_input.value.strip()
        if not loc:
            print("❌ Enter a location.")
            return
        try:
            data = get_weather_data(loc)
            create_temperature_visualisation(data)
        except Exception as e:
            print(f"⚠️ {e}")

def on_precip_clicked(b):
    with output_area:
        clear_output()
        loc = location_input.value.strip()
        if not loc:
            print("❌ Enter a location.")
            return
        try:
            data = get_weather_data(loc)
            create_precipitation_visualisation(data)
        except Exception as e:
            print(f"⚠️ {e}")

# Link buttons
ask_button.on_click(on_ask_clicked)
temp_button.on_click(on_temp_clicked)
precip_button.on_click(on_precip_clicked)

# Layout
ui = widgets.VBox([
    widgets.HTML("<h2>☁️ Weather Assistant</h2>"),
    question_input,
    ask_button,
    widgets.HTML("<hr>"),
    location_input,
    widgets.HBox([temp_button, precip_button]),
    output_area
])

# Display the UI
display(ui)
